In [82]:
import pandas as pd

In [83]:
df = pd.read_csv('~/ufes/datasets/ISIC2024/train-metadata.csv')

/tmp/ipykernel_10783/561533406.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('~/ufes/datasets/ISIC2024/train-metadata.csv')


In [84]:
non_features = ['isic_id', 'target', 'patient_id',
    'image_type', 'tbp_tile_type', 'attribution', 'copyright_license', 
    'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 
    'iddx_4', 'iddx_5', 'mel_mitotic_index', 'mel_thick_mm']

In [85]:
features = list(set(df.columns) - set(non_features))
print(f'Number of features: {len(features)}')

Number of features: 39


In [86]:
df['target'] = df['target'].astype(int)
df['diagnosis'] = df['target'].map({0:'benign', 1:'malignant'}) 
df['diagnosis'] = df['diagnosis'].astype('category')
df['diagnosis'].value_counts()

diagnosis
benign       400666
malignant       393
Name: count, dtype: int64

## Missing data analysis

### Columns with missing values

In [87]:
is_na = df.isna()
count_nan = is_na.sum()
features_with_nan_values = list(count_nan[features][count_nan > 0].index)
count_nan[features_with_nan_values]

age_approx              2798
anatom_site_general     5756
sex                    11517
dtype: int64

### Percentage of missing values

In [88]:
count_nan[features_with_nan_values] / len(df) * 100

age_approx             0.697653
anatom_site_general    1.435200
sex                    2.871647
dtype: float64

### Columns with values equals to zero

In [89]:
is_zero = df.isin([0])
count_zero = is_zero.sum()
features_with_zeroed_values = list(count_zero[features][count_zero > 0].index)
count_zero[features_with_zeroed_values]

tbp_lv_symm_2axis_angle        18562
tbp_lv_norm_color              26182
tbp_lv_color_std_mean          26187
tbp_lv_nevi_confidence             6
tbp_lv_radial_color_std_max    29733
dtype: int64

### Percentage of values equals to zero

In [90]:
count_zero[features_with_zeroed_values] / len(df) * 100

tbp_lv_symm_2axis_angle        4.628247
tbp_lv_norm_color              6.528217
tbp_lv_color_std_mean          6.529463
tbp_lv_nevi_confidence         0.001496
tbp_lv_radial_color_std_max    7.413622
dtype: float64

## Per-class analysis

### Frequency of missing values per class

In [91]:
is_na_per_diagnosis = df[features_with_nan_values + ['diagnosis']].groupby(['diagnosis']).apply(lambda x : x.isna())
for diagnosis in df['diagnosis'].unique():
    print('-' * 50)
    print(f'Percentage of missing values for {diagnosis.upper()} class:')
    print(is_na_per_diagnosis.loc[diagnosis].sum(),'\n')

--------------------------------------------------
Percentage of missing values for BENIGN class:
age_approx              2795
anatom_site_general     5756
sex                    11507
diagnosis                  0
dtype: int64 

--------------------------------------------------
Percentage of missing values for MALIGNANT class:
age_approx              3
anatom_site_general     0
sex                    10
diagnosis               0
dtype: int64 



/tmp/ipykernel_10783/3569823170.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  is_na_per_diagnosis = df[features_with_nan_values + ['diagnosis']].groupby(['diagnosis']).apply(lambda x : x.isna())
/tmp/ipykernel_10783/3569823170.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  is_na_per_diagnosis = df[features_with_nan_values + ['diagnosis']].groupby(['diagnosis']).apply(lambda x : x.isna())


### Frequency of values equals to zero per class

In [92]:
is_na_per_diagnosis = df[features_with_zeroed_values + ['diagnosis']].groupby(['diagnosis']).apply(lambda x : x.isin([0]))
for diagnosis in df['diagnosis'].unique():
    print('-' * 50)
    print(f'Percentage of missing values for {diagnosis.upper()} class:')
    print(is_na_per_diagnosis.loc[diagnosis].sum(),'\n')

/tmp/ipykernel_10783/3069690140.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  is_na_per_diagnosis = df[features_with_zeroed_values + ['diagnosis']].groupby(['diagnosis']).apply(lambda x : x.isin([0]))


--------------------------------------------------
Percentage of missing values for BENIGN class:
tbp_lv_symm_2axis_angle        18543
tbp_lv_norm_color              26092
tbp_lv_color_std_mean          26097
tbp_lv_nevi_confidence             6
tbp_lv_radial_color_std_max    29642
diagnosis                          0
dtype: int64 

--------------------------------------------------
Percentage of missing values for MALIGNANT class:
tbp_lv_symm_2axis_angle        19
tbp_lv_norm_color              90
tbp_lv_color_std_mean          90
tbp_lv_nevi_confidence          0
tbp_lv_radial_color_std_max    91
diagnosis                       0
dtype: int64 



/tmp/ipykernel_10783/3069690140.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  is_na_per_diagnosis = df[features_with_zeroed_values + ['diagnosis']].groupby(['diagnosis']).apply(lambda x : x.isin([0]))
